In [1]:
import pandas as pd
from datasets import Dataset, DatasetDict,load_from_disk,load_dataset

In [2]:
dataset_path = "medal_dataset"
dataset = load_from_disk(dataset_path)

# dataset = load_dataset("lutful2004/MeDAL-dataset")


In [3]:
from datasets import ClassLabel
import numpy as np

# Combine all splits into a single dataset
combined_data = {
    'ABSTRACT_ID': [],
    'TEXT': [],
    'LOCATION': [],
    'LABEL': []
}

for split in ['train', 'test', 'valid']:
    for key in combined_data.keys():
        combined_data[key].extend(dataset[split][key])

combined_dataset = Dataset.from_dict(combined_data)

# Convert LABEL to ClassLabel feature
labels = combined_dataset.unique('LABEL')
label_feature = ClassLabel(names=labels)
combined_dataset = combined_dataset.cast_column('LABEL', label_feature)

# Create datasets with different thresholds
thresholds = {
    '10percent': 0.1,
    '5percent': 0.05,
    '1percent': 0.01
}

datasets_by_threshold = {}

for name, threshold in thresholds.items():
    # Calculate number of samples for each split
    total_size = len(combined_dataset)
    train_size = int(total_size * threshold * 0.8)
    valid_size = int(total_size * threshold * 0.1)
    test_size = int(total_size * threshold * 0.1)
    
    # Create stratified splits
    stratified_dataset = combined_dataset.train_test_split(
        test_size=valid_size+test_size,
        stratify_by_column='LABEL'
    )
    
    test_valid = stratified_dataset['test'].train_test_split(
        test_size=test_size/(valid_size+test_size),
        stratify_by_column='LABEL'
    )
    
    # Create the dataset dictionary
    datasets_by_threshold[name] = DatasetDict({
        'train': stratified_dataset['train'],
        'valid': test_valid['train'],
        'test': test_valid['test']
    })
    
    print(f"{name} dataset:")
    print(f"  Train: {len(datasets_by_threshold[name]['train'])} samples")
    print(f"  Valid: {len(datasets_by_threshold[name]['valid'])} samples")
    print(f"  Test: {len(datasets_by_threshold[name]['test'])} samples")

# Save the datasets
for name, dataset_dict in datasets_by_threshold.items():
    dataset_dict.save_to_disk(f"medal_dataset_{name}")

MemoryError: 